In [1]:
!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-06lmazbg
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-06lmazbg
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done
  Using cached torch-1.10.1-cp38-cp38-manylinux1_x86_64.whl.metadata (24 kB)
Using cached torch-1.10.1-cp38-cp38-manylinux1_x86_64.whl (881.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.10.1 which is incompatible.


In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [2]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [3]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
device = torch.device("cuda:0")

In [5]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /home/cho/test/.cache/kobert_v1.zip
using cached model. /home/cho/test/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/home/cho/anaconda3/envs/KoBert/lib/python3.8/site-packages/transformers/modeling_utils.py:1211: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolv

In [6]:
import pandas as pd
from io import StringIO

train = pd.read_csv('/home/cho/AIFFEL_quest_rs/GoingDeeper/Go01/raw_data.csv')
tsv_train = train.to_csv(sep='\t', index=False)

tsv_df = pd.read_csv(StringIO(tsv_train), sep="\t")
# \n을 공백으로 바꿔서 conversation 컬럼 처리
tsv_df['conversation'] = tsv_df['conversation'].replace(r'\n', ' ', regex=True)
print(tsv_df['conversation'])

# 수정된 데이터를 파일로 저장
tsv_df.to_csv('train.tsv', sep='\t', index=False)


dataset_train = nlp.data.TSVDataset('./train.tsv',field_indices=[2,3], num_discard_samples=1)

0       지금 너 스스로를 죽여달라고 애원하는 것인가?  아닙니다. 죄송합니다.  죽을 거면...
1       길동경찰서입니다. 9시 40분 마트에 폭발물을 설치할거다. 네? 똑바로 들어 한번만...
2       너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어. 그만해. 니들 놀리는거 재미없...
3       어이 거기 예?? 너 말이야 너. 이리 오라고 무슨 일. 너 옷 좋아보인다? 얘 돈...
4       저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
                              ...                        
4841    요즘 나는 축구 배워보고 싶다 키키 축구? 축구 룰에 대해서는 알아? 오오 알지 키...
4842    프로게이머들은 좋아하는 거 하고 돈 버니까 좋겠다 키키 부럽. 임요한 진짜 돈 많이...
4843    너는 해외여행 자주 다니는 편이야? 키키 아니. 비행기 타는 걸 안 좋아해서 해외여...
4844    겨울에 눈이 제발 적게 오면 좋겠다 나도 키키 어릴 땐 눈 오는 게 되게 좋아했었는...
4845    요즘은 비나 태풍이 한번 와도 피해가 큰 듯 어 ㅠㅠ 나는 아직 태풍 매미를 잊지 ...
Name: conversation, Length: 4846, dtype: object


In [7]:
test = pd.read_csv('/home/cho/AIFFEL_quest_rs/GoingDeeper/Go01/augmented_replaced_text_data.csv')
tsv_test = train.to_csv(sep='\t', index=False)

tsv_df = pd.read_csv(StringIO(tsv_test), sep="\t")
# \n을 공백으로 바꿔서 conversation 컬럼 처리
tsv_df['conversation'] = tsv_df['conversation'].replace(r'\n', ' ', regex=True)
print(tsv_df['conversation'])

# 수정된 데이터를 파일로 저장
tsv_df.to_csv('test.tsv', sep='\t', index=False)


dataset_test = nlp.data.TSVDataset('./test.tsv',field_indices=[2,3], num_discard_samples=1)

0       지금 너 스스로를 죽여달라고 애원하는 것인가?  아닙니다. 죄송합니다.  죽을 거면...
1       길동경찰서입니다. 9시 40분 마트에 폭발물을 설치할거다. 네? 똑바로 들어 한번만...
2       너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어. 그만해. 니들 놀리는거 재미없...
3       어이 거기 예?? 너 말이야 너. 이리 오라고 무슨 일. 너 옷 좋아보인다? 얘 돈...
4       저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
                              ...                        
4841    요즘 나는 축구 배워보고 싶다 키키 축구? 축구 룰에 대해서는 알아? 오오 알지 키...
4842    프로게이머들은 좋아하는 거 하고 돈 버니까 좋겠다 키키 부럽. 임요한 진짜 돈 많이...
4843    너는 해외여행 자주 다니는 편이야? 키키 아니. 비행기 타는 걸 안 좋아해서 해외여...
4844    겨울에 눈이 제발 적게 오면 좋겠다 나도 키키 어릴 땐 눈 오는 게 되게 좋아했었는...
4845    요즘은 비나 태풍이 한번 와도 피해가 큰 듯 어 ㅠㅠ 나는 아직 태풍 매미를 잊지 ...
Name: conversation, Length: 4846, dtype: object


In [8]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [9]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/cho/test/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [10]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [11]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [12]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [13]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [14]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [15]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [16]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [17]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [18]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [19]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [20]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/76 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.7605568170547485 train acc 0.125
epoch 1 train acc 0.4146792763157895


  0%|          | 0/76 [00:00<?, ?it/s]

epoch 1 test acc 0.20929276315789475


  0%|          | 0/76 [00:00<?, ?it/s]

...

epoch 4 test acc 0.5929723255148742


  0%|          | 0/76 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 1.7423129081726074 train acc 0.28125
epoch 5 train acc 0.5172250429061784


  0%|          | 0/76 [00:00<?, ?it/s]

epoch 5 test acc 0.6083470394736842


In [ ]:
def inference(model, tokenizer, vocab, sentence, device, max_len=64):
    model.eval()  # 평가 모드 설정

    # 문장 토큰화
    tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
    transform = nlp.data.BERTSentenceTransform(tok, max_seq_length=max_len, pad=True, pair=False)
    
    # 토큰 변환
    tokenized_sentence = transform([sentence])
    token_ids = torch.tensor(tokenized_sentence[0]).unsqueeze(0).long().to(device)
    valid_length = torch.tensor(tokenized_sentence[1]).unsqueeze(0).long().to(device)
    segment_ids = torch.tensor(tokenized_sentence[2]).unsqueeze(0).long().to(device)

    # 모델 예측
    with torch.no_grad():
        output = model(token_ids, valid_length, segment_ids)
        predicted_class = torch.argmax(output, dim=1).cpu().numpy()[0]

    return predicted_class


sentence = '너가 민수라는 남자 제자니? 네 맞는데요 왜 그러시죠? 혹시? 무슨 일은 무슨 일은 그냥 선생님 콩팥팥 정도 가져가려고 네.? 장난치시지 마세요. 장난이 너무 심하신 거 아니신가요? 너무 어린애라 그런지 상황 판단 못하네. 보여 줄 테니 봐봐 장난인지 아닌지 어.? 왜 그러시는 건가요 신고 할 건데 당장 신고 할 거예요. 신고해보던지 어딘지는 알고 신고하게? 저한테 물어볼 거 있으시면 최대한 할게요 선생님 해치지 말아주세요. 내가 말하는 거 보여 주기만 할게요 그러면 그냥 깔끔하게 끝날 일이야 그렇게 바로 할게요 바로 할게요'

predicted_label = inference(model, tokenizer, vocab, sentence, device)

# 예제 문장 예측
print(f"예측된 클래스: {predicted_label}") #협박 대화 0


예측된 클래스: 3
